## Base Funcs

In [158]:
import pandas as pd
import jieba
import itertools
from tqdm.auto import tqdm
import json
from collections import defaultdict



In [159]:
# !pip install jieba
# !pip install emoji
import jieba
def segmentate_sentence(sent):
    """
    word segmentation with jieba
    ref : https://github.com/fxsjy/jieba
    Args :
        sent(Str) : a sentence without segmentation, e.g "Iloveyou"
    Output :
        sent(str) : segmentated sentence, e.g "I love you"
    """
    # sent = clean(sent)
    seg_list = jieba.lcut(sent, cut_all=False)
    
    seg_list = [i for i in seg_list if i!=" "]
    return seg_list
    
def clean(x):
    """clean tweets
    :param: a tweet (str)
    :return a preprocessed tweet (str)
    """
    import emoji
    from collections import Counter
    import re
    emojis = ''.join(emoji.UNICODE_EMOJI.keys())
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
    username_pattern = re.compile(
        r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)")
    url_pattern = re.compile(
        r"[-a-zA-Z0-9@:%_\+.~#?&//=]{2,256}\.[a-z]{2,4}\b(\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?")

    chinese_pattern=re.compile(r"[^\u4e00-\u9fa5]+")
#     x = username_pattern.sub('', x)
#     x = url_pattern.sub('', x)
    x = chinese_pattern.sub('', x)
#     x = re.sub(r'[0-9]+', '', x)
#     table = x.maketrans({ '/': ' ',  '#': ' ', '@':' ', ':':' '})
#     x = x.translate(table)
    x = re.sub(pattern, '', x)
#     x = re.sub(r"[^a-zA-Z0-9]","",x) # 특수문자 제거
    x = x.strip()
    return x
    

## Jieba

In [246]:
#from xlsx to csv
def make_user_dict(xlsx_path, path) :
    """
    make a custom dictionary for jieba
    It makes a txt file to communicate with jieba
    xlsx_path : A path of dictionary
    path : A result path of dictionary
    """
    df = pd.read_excel(xlsx_path)
    with open(path, "w") as f :
        f.writelines('\n'.join(df.loc[:,'words']))
    f.close()
    
def add_words(raw_paths, existing_data=None) :
    """
    add new data with existing data
    input :
        raw_paths : list of path in train_data folder
        existing_data : dataframe - existing data, if not given, it initalize the dataset
    output :
        set of words
    """
    # # !pip install googletrans==3.1.0a0
    # # ref) https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group
    # # ref) https://pypi.org/project/googletrans/
    # from googletrans import Translator
    # gmt = Translator()
    # gmt.translate(word, dest='ko').text

    bag_of_words = []
    cnt_words = []
    meanings = []
    positions = []

    if existing_data is not None :
        bag_of_words = list(existing_data.loc[:,'words'])
        cnt_words = list(existing_data.loc[:,'cnt'])
        meanings = list(existing_data.loc[:,'meaning'])
        positions = list(existing_data.loc[:,'position'])

    for path in raw_paths :
        df = pd.read_csv(f"train_data/{path}.csv")

        for row in tqdm(range(df.shape[0])) :
            for word in segmentate_sentence(df['English title'][row]) :
                if word in bag_of_words :
                    cnt_words[bag_of_words.index(word)]+=1
                else :
                    bag_of_words.append(word)
                    cnt_words.append(1)
                    res = translate(word)
                    meanings.append(json.loads(res)['message']['result']['translatedText'])
                    positions.append("")

    df = pd.DataFrame(list(zip(bag_of_words, meanings, positions, cnt_words)),columns =['words', 'meaning', 'position','cnt'])
    return df.sort_values(by='cnt', ascending=False).reset_index(drop=True)

def translate(target) :
    import os
    import sys
    import urllib.request
    # from Local import client_id, client_secret
    client_id = "sy3GuTVVRhzvp2xC8vVM" # 개발자센터에서 발급받은 Client ID 값
    client_secret = "D1CqkDNhAq" 
    encText = urllib.parse.quote(target)
    data = "source=zh-CN&target=ko&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        return response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

def retrain(txt_path, ref_path) :
    """
    update the dictionary with txt file.
    if line startswith '`' : update with this
    input :
        txt_path : updated contents
        ref_path : reference dictionary's path
    """
    print("#####Retrain Started#####")
    dic = defaultdict(str)
    reference= pd.read_excel(ref_path)
    reference = reference.set_index('words', drop=True)
    changed=[]
    with open(txt_path, "r") as f:
        li = f.readlines()
        li = [i.strip() for i in li]

    for word in li :
        if word.startswith('---') or word.startswith('result'):
            continue
        if word.startswith('₩'):
            try :
                key, item = word.split(',')
                dic[key.strip()[1:]] = item.strip()
            except :
                pass

    for key in dic.keys() :
        print(key)
        if key in reference.index :
            if reference.loc[key, 'meaning'] != dic[key] :
                reference.loc[key, 'meaning'] = dic[key]
                changed.append(key)
        else :
            reference.loc[key] = [dic[key], 0,1]
            changed.append(key)
    for key in changed :            
        print(f"{key} -> {reference.loc[key]['meaning']}")

    print(f"data has been updated {len(changed)} times")
    reference.to_excel(ref_path)  
    print("#####Retrain Ended#####")


In [254]:
##Mark Position##
reference= pd.read_excel(train_data)
reference = reference.set_index('words', drop=True)
reference = reference.dropna(axis=0)
reference


,meaning,position,cnt
words,,,
新款,신상,9,232
2022,2022,9,200
女,여성,5,189
上衣,상의,8,110
夏季,여름,9,101
...,...,...,...
生活在左,A Life On the Left,0,1
植物,천연,0,1
染,염색,0,1


In [283]:
#######Add position of words which are in the given set#####
def give_position(set_of_words, position, reference) :
    """
    Add position information to reference file
    """
    for idx in reference.index :
        word = reference.loc[idx,'meaning']
        word = str(word)
        flag=False
        for chaidx in range(len(str(word))) :
            if word[chaidx] in set_of_words :
                flag=True
            if chaidx != len(word)-1 : ### You should renew here
                if word[chaidx:chaidx+2] in set_of_words :
                    flag=True
        if flag :
            reference.loc[idx,'position'] = position
            print(idx, reference.loc[idx,'meaning'], "-->", position)
    
    return reference

set_of_words = ['봄','여름','가을','겨울']
position = 11
reference= pd.read_excel(train_data)
reference = reference.set_index('words', drop=True)
reference = reference.dropna(axis=0)
reference = give_position(set_of_words, position, reference)

夏季 여름 --> 11
春季 봄철 --> 11
早春 이른 봄 --> 11
春秋 봄/가을 --> 11
春装 봄옷 --> 11
春夏 봄/여름 --> 11
夏 여름 --> 11
春 봄 --> 11
秋冬 가을/겨울 --> 11
春款 봄 스타일 --> 11
冬季 겨울 --> 11
秋 가을 --> 11
夏日 여름 --> 11
春秋季 봄/가을 --> 11
春天 봄 --> 11
秋季 가을철 --> 11
春夏季 봄여름 --> 11
冬装 겨울옷 --> 11
显瘦春 날씬해 보이는 봄 --> 11
夏大码 여름 빅 사이즈 --> 11
冬 겨울 --> 11
夏天 여름 --> 11
秋装 가을 옷차림 --> 11
初秋 초가을 --> 11


In [290]:
##Start Working##
def forward(data_path, train_data, user_dict, start) :
    df = pd.read_csv(data_path)
    #########LOAD TRAINED SECTION##########
    make_user_dict(train_data, user_dict)
    jieba.load_userdict(user_dict) # load customized dictionary
    reference= pd.read_excel(train_data)
    reference = reference.set_index('words', drop=True)
    reference = reference.dropna(axis=0)
    #########LOAD TRAINED SECTION##########
    with open("./working/auto.txt", "w") as h :
        with open("./working/new_data.txt", "w")as g :
            for row in tqdm(range(start, df.shape[0])) :
                ###################Generating Section###########################
                sentence = segmentate_sentence(df['English title'][row])
                translated = []
                buff = defaultdict(str)
                key_words= ['2022', '신상', '2022신상', '22', '20', '년'] ## For Extra section
                season=""
                new_flag=False
                for word in segmentate_sentence(df['English title'][row]) :
                    if word in reference.index :
                        if reference.loc[word, 'position'] ==11 : ##Check Season
                            season = reference.loc[word,'meaning']
                        buff[word] = list(reference.loc[word, :])
                        translated.append(str(buff[word][0]))
                    else :
                        res = translate(word)
                        res = json.loads(res)['message']['result']['translatedText']
                        # res = word
                        buff[word] = [res,0,1]
                        translated.append(buff[word][0])
                ###################Generating Section###########################

                ###################Extra Section###########################
                    if new_flag==False :
                        if buff[word][0] in key_words :        
                            new_flag=True

                translated = [word for word in translated if (word not in key_words)]
                ###################Extra Section###########################

                ###################Writing Section###########################
                h.write(f"{row+1}.\n")
                h.write(f"{' '.join(sentence)}\n")
                h.write(f"{' '.join(translated)}, ")
                if new_flag :
                    if season == "" :
                        h.write(f"2022 신상\n")
                    else :
                        h.write(f"2022 {season} 신상\n")
                else :
                    h.write("\n")
                g.write(f"\n-------------------{row+1}------------\n")
                for key, item in buff.items() :
                    g.write(f"{key}, {item[0]}\n")
                ###################Writing Section###########################
        g.close()
        print("auto.txt has successfully saved as ./working/auto.txt")
    h.close()
    print("new_data.txt has successfully saved as ./working/new_data.txt")
    print("Everything has completed, Go to work")

data_path="working/work.csv"
train_data="./train_data/reference.xlsx"
user_dict="./train_data/userdict.txt"

retrain("./working/new_data.txt", "./train_data/reference.xlsx")
forward(data_path, train_data, user_dict, 0)

#####Retrain Started#####
月牙
大连东越
逃离
地球
御姐
卡农
月牙 -> Yueya
大连东越 -> Dalian-dongyue
逃离 -> TAOLI
地球 -> DIQIU
御姐 -> 걸크러쉬
卡农 -> Qiaonong
data has been updated 6 times
#####Retrain Ended#####


 18%|█▊        | 18/100 [00:12<00:35,  2.34it/s]

In [249]:
reference

,meaning,position,cnt
words,,,
新款,신상,9,232
2022,2022,9,200
女,여성,5,189
上衣,상의,8,110
夏季,여름,9,101
...,...,...,...
好看,스타일 좋은,0,1
纯欲,순수하면서도 섹시한,0,1
生活在左,A Life On the Left,0,1


## Generating - word

One hot encoding including position infos.

meaning, category, position info

1. give main meaning
2. add meaning
3. add position meaning

words - word - meaning - cate - 
1
2
3
4

In [ ]:
###Opn other files and save on this table too###
###No overlap